# Crowdsouring-Daten nutzen

Daten aus OpenStreetMap (OSM) und WikiData zusammen mit städtischen Daten nutzen

**Jupyter-Notebook laden: [bit.ly/gisforum2021](https://bit.ly/gisforum2021)**

## Initialisierung

In [2]:
# Python Pakete installieren (auf JupyterHub nicht nötig!)
# pip install -r requirements.txt

In [ ]:
import os
import json
from pprint import pprint

import requests
import folium
import geopandas
import pandas as pd
from IPython.display import HTML, display

import utils

In [ ]:
lv95 = 'EPSG:2056'
wgs84 = 'EPSG:4326'

# Städtische Daten «Kunst im öffentlichen Raum (KiöR)»

Im ersten Teil schauen wir uns an, wie wir städtische Daten mit Daten aus OpenStreetMap (OSM) anreichern können.
Dazu laden wir uns zuerst den Datensatz «Kunst im öffentlichen Raum» via WFS.

[Via OGD-Portal](https://data.stadt-zuerich.ch/dataset?q=kunst+im+%C3%B6ffentlichen+raum) finden wir den Datensatz [Kunst im Stadtraum](https://data.stadt-zuerich.ch/dataset/geo_kunst_im_stadtraum).
Von dort via GeoJSON auf das [Geoportal](https://www.stadt-zuerich.ch/geodaten/download/Kunst_im_Stadtraum?format=10009) zur richtigen URL.

## Variablen setzen

In [ ]:
kioer_geojson_url = 'https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Kunst_im_Stadtraum?service=WFS&version=1.1.0&request=GetFeature&outputFormat=GeoJSON&typename=view_kioer'
kioer_layer = 'view_kioer'
wfs_url = 'https://www.ogd.stadt-zuerich.ch/wfs/geoportal/Kunst_im_Stadtraum' 
layers = utils.get_layers_from_wfs(wfs_url)

## WFS GetFeature Request absetzen

In [ ]:
r = requests.get(wfs_url, params={
    'service': 'WFS',
    'version': '1.0.0',
    'request': 'GetFeature',
    'typename': kioer_layer,
    'outputFormat': 'GeoJSON'
})

kioer_geo = r.json()
kioer_geo

# Daten aus OpenStreetMap laden

Wir suchen thematisch passende Daten aus OpenStreetMap.
Um diese zu finden, lohnt es sich das [OpenStreetMap Wiki](https://wiki.openstreetmap.org/wiki/Main_Page) zu durchsuchen und passende Tags zu finden.

**ACHTUNG:** Daten aus OpenStreetMap stehen unter der [Open Data Commons Open Database License (ODbL)](https://opendatacommons.org/licenses/odbl/1-0/). D.h. die Verwendung der Daten ist erlaubt wenn:

- die Quelle erwähnt wird und
- allfällige Änderungen unter derselben Lizenz zur Verfügung stehen

## OSM-Daten laden

Daten von OpenStreetMap (OSM) können u.a. via Overpass API geladen werden.
Overpass hat eine eigene Abfragesprache ([Overpass QL](https://wiki.openstreetmap.org/wiki/Overpass_API/Overpass_QL)), mit der Objekte (Nodes, Ways, Relations) abgefragt werden können.

![OpenStreetMap Datenmodell](assets/images/osm_data_model.png)
<sup>[Bildquelle: &copy; Itinero](https://docs.itinero.tech/docs/osmsharp/osm.html)</sup>

## Overpass Query absetzen

[Query in Overpass Turbo ausführen](https://osm.li/UfK)

In [ ]:
artwork_zh = """
/*
Alle Kunstwerke (tourism=artwork) in der Stadt Zürich
*/
[out:json];
area["name"="Zürich"]["wikipedia"="de:Zürich"]->.perimeter; 
(
  nwr[tourism=artwork](area.perimeter);
);
out center;
"""
osm_result = utils.overpass_query(artwork_zh)
osm_result

## Karte mit den Resultaten

In [ ]:
# Basiskarte
m = utils.base_map()

# KiöR-Daten hinzufügen
kioer_layer = folium.FeatureGroup(name='KiöR', show=True)
utils.style_layer(kioer_geo, kioer_layer, icon_color='#031cff', icon='certificate', prefix='fa')
kioer_layer.add_to(m)

# OSM-Daten hinzufügen
osm_layer = folium.FeatureGroup(name='OSM: tourism=artwork', show=True)
utils.style_layer(osm_result, osm_layer, icon_color='#ff0000', icon='fire', prefix='fa')
osm_layer.add_to(m)

# Add controls for layers
folium.LayerControl().add_to(m)
m

## Spatial Join der zwei Quellen

Um die beiden Quellen zu matchen gibt es grundsätzlich zwei Möglichkeiten:

1. Join über Attribute (z.B. Titel des Kunstwerks)
1. Räumlicher Join (Objekte die nah beieinander sind)

Oder eine Kombination der beiden Ansätze.

Im folgenden schauen wir uns den **Räumlichen Join («Spatial Join»)** genauer an.

### GeoDataFrame für geopandas erstellen

In [ ]:
kioer_df = geopandas.GeoDataFrame.from_features(kioer_geo, crs=wgs84)
kioer_df.head()

In [ ]:
osm_df = geopandas.GeoDataFrame.from_features(osm_result, crs=wgs84)
osm_df.head()

### Geometrien für Join vorbereiten

In [ ]:
# Einträge mit leerer Geometrie
kioer_df[(pd.isna(kioer_df.geometry))].head()

In [ ]:
# Alle Einträge mit leerer Geometrie entfernen
kior_buf = kioer_df.dropna(subset=['geometry']).reset_index(drop=True)
osm_buf = osm_df.dropna(subset=['geometry']).reset_index(drop=True)

# CRS zu LV95 re-projezieren
kior_buf = kior_buf.to_crs(lv95) # Konvertieren zu 2D Koordinatensystem
osm_buf = osm_buf.to_crs(lv95) # Konvertieren zu 2D Koordinatensystem

# Buffer um die Geometrien hinzufügen (10 Meter)
kior_buf['geometry'] = kior_buf.geometry.buffer(10) 
osm_buf['geometry'] = osm_buf.geometry.buffer(10)

### Vorschau der Geometrien auf der Karte

In [ ]:
# Basiskarte 
bm = utils.base_map()

folium.features.GeoJson(
    kior_buf.to_crs(wgs84).to_json(),
    tooltip=folium.features.GeoJsonTooltip(
        fields=['titel', 'autoren', 'datierung', 'material'],
        aliases=['Titel:', 'Künstler:', 'Datierung:', 'Material:'],    
    )
).add_to(bm)

folium.features.GeoJson(
    osm_buf.to_crs(wgs84).to_json(),
    style_function=lambda x: {'fillColor': '#FF0000', 'color': '#FF0000'},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name', 'artist_name', 'wikidata'],
        aliases=['Titel:', 'Künstler:', "Wikidata:"],                      
    )
).add_to(bm)
bm

### Join durchführen

In [ ]:
# spatial join über die beiden Geometrien
sjoin_kunst = geopandas.sjoin(kior_buf, osm_buf, how='left', predicate='intersects', lsuffix='kior', rsuffix='osm').reset_index()

# Wie zurück zu WGS84
sjoin_kunst = sjoin_kunst.to_crs(wgs84)


with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(sjoin_kunst[['titel', 'name', 'autoren', 'artist_name', 'material_kior', 'material_osm', 'datierung']])

### Resultat des Joins auf der Karte

In [ ]:
folium.features.GeoJson(
    sjoin_kunst.to_json(),
    style_function=lambda x: {'fillColor': '#09bd63', 'color': '#09bd63'},
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name', 'artist_name', 'wikidata'],
        aliases=['Titel:', 'Künstler:', "Wikidata:"],                      
    )
).add_to(bm)
bm

### OSM-Einträge ohne Match

In [ ]:
# OSM-Einträge ohne Match
osm_no_match = osm_df[(~osm_df.id.isin(sjoin_kunst.id))].reset_index()
osm_no_match.head()

### Geopackage erstellen

Das Result lässt sich sehr einfach als Geopackage exportieren.

Im Beispiel ein Layer für die gematchten und ein Layer für die nicht-gematchten Einträge.

**Das Geopackage lässt sich anschliessend z.B. in QGIS weiterverarbeiten**

In [ ]:
osm_no_match.to_file(os.path.join('..', 'kunst_package.gpkg'), layer='osm_no_match', driver="GPKG")
sjoin_kunst.to_file(os.path.join('..', 'kunst_package.gpkg'), layer='kioer_osm_match', driver="GPKG")

# Daten aus WikiData laden

WikiData ist ein Schwesterprojekt von Wikipedia.
Es beinhaltet strukturierte Daten, welche via API abgefragt werden können (REST oder SPARQL).

Alle Daten aus WikiData sind [CreativeCommons-Zero (CC0) lizenziert](https://creativecommons.org/publicdomain/zero/1.0/deed.de), können also bedenkenlos weiterverwendet werden.

## Daten via WikiData-Verweise von OSM holen

[OpenStreetMap Node «Heureka»](https://www.openstreetmap.org/node/268472148)


In [ ]:
# WikiData Verweise von OpenStreetMap
osm_kunst_wd = osm_df[(~pd.isna(osm_df.wikidata))] # alle Einträge, bei denen "wikidata" nicht leer ist
osm_kunst_wd = osm_kunst_wd[['geometry', 'type', 'id', 'artist_name', 'artist:wikidata', 'name', 'wikidata']].reset_index(drop=True)
osm_kunst_wd.head()

### Beispiel-Abfrage nach WikiData Item

[Wikidata-Item «Heureka»](https://www.wikidata.org/wiki/Q1378316)

In [ ]:
wikidata_item = utils.wikidata_item(osm_kunst_wd['wikidata'].iloc[0])
pprint(wikidata_item['sitelinks'])

**Bilder aus Wikimedia Commons laden:**

In [ ]:
# Bilder laden
category = wikidata_item['sitelinks']['commonswiki']['title']
urls = utils.images_from_commons_category(category, image_size=500)
urls

In [ ]:
#Bilder anzeigen
display(HTML(''.join([utils.img_html(url) for url in urls])))

## Daten via SPARQL aus WikiData beziehen

Um Daten aus WikiData zu laden, können Abfragen mit SPARQL gemacht werden. SPARQL ist eine SQL-ähnliche Abfragesprache für Linked Data.

Die Idee von Linked Data ist es, einen Informations-Graphen in Form von sogenannten «Triples» abzubilden:

![Linked Data Triple](assets/images/triple.png)


**Triple = Subjekt (z.B. Marie) -> Prädikat (z.B. birthPlace) -> Objekt (z.B. Italy)**

<sup>[Bildquelle: &copy; WordLift](https://wordlift.io/blog/en/entity/linked-data/)</sup>

### SPARQL-Query absetzen

[Link zum Wikidata Query Service](https://w.wiki/4Cep)

In [ ]:
# direkte SPARQL-Query auf WikiData
wd_kunstwerke_query = """
SELECT DISTINCT ?artwork ?artworkLabel ?creator ?creatorLabel ?creatorBirthday ?createDateOfDeath ?geo
WHERE
{
  ?artwork wdt:P136 wd:Q557141 .           # Genre "Kunst im öffentlichen Raum"
  ?artwork wdt:P131 wd:Q72 .               # liegt in Zürich
  OPTIONAL {
    ?artwork wdt:P170 ?creator .           # Urheber des Werks
    ?creator wdt:P569 ?creatorBirthday .   # Geburtsdatum des Urhebers
    ?creator wdt:P570 ?createDateOfDeath . # Todesdatum des Urhebers
  }   
  OPTIONAL {
    ?artwork wdt:P625 ?geo .               # Koordinaten für Kartenansicht
  }       
 
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de,en"
  }
}
ORDER BY ?artworkLabel
"""
wd_result = utils.wikidata_query(wd_kunstwerke_query)
wikidata_df = pd.DataFrame(wd_result)
wikidata_df.head()

### Result als GeoDataFrame

In [ ]:
wikidata_df[(pd.isna(wikidata_df.geo))]

In [ ]:
# remove entries with empty coordinates
wikidata_df = wikidata_df.dropna(subset=['geo']).reset_index(drop=True)
wikidata_df.head()

In [ ]:
# Erstelle GeoDataFrame
#wikidata_gdf = geopandas.GeoDataFrame(wikidata_df, geometry=geopandas.points_from_xy(wikidata_df.lon, wikidata_df.lat))
wikidata_gdf = geopandas.GeoDataFrame(wikidata_df, geometry=geopandas.GeoSeries.from_wkt(wikidata_df.geo))
wikidata_gdf = wikidata_gdf.drop(columns=['geo'])
wikidata_gdf = wikidata_gdf.set_crs(wgs84)
wikidata_gdf = wikidata_gdf.dropna(subset=['geometry']).reset_index(drop=True)
wikidata_gdf.head()

### Resultat auf Karte darstellen

In [ ]:
# Basiskarte 
wikimap = utils.base_map()

wd_layer = folium.FeatureGroup(name='WikiData: public art', show=True)
utils.style_layer(wikidata_gdf, wd_layer, color='green', icon='eye', prefix='fa')
wd_layer.add_to(wikimap)

wikimap

# Food for thought / Aufgaben für Fortgeschrittene

1. Wie können wir einen Spatial Join zwischen dem KiöR und dem WikiData Dataset machen?
1. Wie kann eine Kombination aus Spatial Join und Attribute Join gemacht werden (um die Qualität der Matches zu verbessern)?
1. Wie könnten Daten aus OSM und WikiData in städtische Daten integriert werden?
    * Als Datenquelle?
    * Als Plausibilitätscheck?
    * Daten abgleichen?
    * ...?
1. Wie könnten städtische Daten in OSM und WikiData integriert werden?
    * via OGD?
    * direkt via Bot?
    * ...?

# Anhang und Tutorials

## OpenStreetMap (OSM)

* [OpenStreetMap Wiki](https://wiki.openstreetmap.org/wiki/DE:Hauptseite)
    * [tourism=artwork](https://wiki.openstreetmap.org/wiki/DE:Tag:tourism%3Dartwork)
    * [Taginfo tourism=artwork](https://taginfo.openstreetmap.org/tags/tourism=artwork#overview)
* [Overpass API](https://wiki.openstreetmap.org/wiki/Overpass_API): Abfragen für OpenStreetMap
    * [Overpass Turbo Weltweit](https://overpass-turbo.eu/)
    * [Overpass Turbo Schweiz](https://overpass-turbo.osm.ch/)
* [OpenSchoolMaps](https://openschoolmaps.ch/): Unterrichtsmaterial inkl. Einsteigerhilfe zu OpenStreetMap
* [learnOSM](https://learnosm.org/de/): Schritt-für-Schritt-Anleitungen

## WikiData

* [WikiData Query Service](https://query.wikidata.org/): Tool für SPARQL-Abfragen inkl. vielen Beispielen
* [WikiData Training](https://www.wikidata.org/wiki/Wikidata:Training): Einführung in WikiData
* [WikiData SPARQL Tutorial](https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial)
* [Wikidata Jupyter Notebooks von OpenDataZurich](https://opendatazurich.github.io/wikidata-training/): MediaWiki API, SPARQL, Bots
* [EUCLID: Querying Linked Data](https://euclid-project.eu/modules/course2.html): SPARQL-Kurs